In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u
from IPython.display import display, HTML
import pandas as pd
import numpy as np

# import observations of selected objects outputted midway from sorcha
# (from PPLinkingFilter.py, save obsv (line 56) to a csv and use that)
obs_df = pd.read_csv('/Users/josephmurtagh/Downloads/midobs1000tnos(1).csv')
obs_df = obs_df.sort_values(by=['ssObjectId'])
obs_df = obs_df.reset_index(drop=True)
obs_df['ssObjectId'] = obs_df['ssObjectId'].astype(str)

# NEXT TWO BLOCKS ARE SETTING UP ARRAY FOR PROCESSING
# convert to record array for sorcha compatability (same process as 
# PPLinkingFilter employs)
nameLen = obs_df['ssObjectId'].str.len().max()
obsv = obs_df.to_records(
    index=False,
    column_dtypes=dict(ssObjectId=f'a{nameLen}', diaSourceId='u8', midPointTai='f8', ra='f8', decl='f8')
)

# split record array by object
i = np.argsort(obsv['ssObjectId'], kind='stable')
ssObjects, idx = np.unique(obsv['ssObjectId'][i], return_index=True)
objSplits = np.split(i, idx[1:])

tracklets = []
a=0
no = 0

# START TRACKLET FINDING LOOP
# select one object at a time to see if it gets linked
for a in range(len(ssObjects)):
    testdf = obs_df.loc[objSplits[a],:]
    testdf = testdf.sort_values(by=['midPointTai'])
    # display(HTML(testdf.to_html()))

    # split by each unique mjd date of observation and put into nights
    splits = np.split(testdf['midPointTai'].values, np.where(np.abs(np.diff(testdf['midPointTai'].values)) >= 0.5)[0] + 1) # splitting observation BY NIGHT (e.g. mjd 60514.9 and 60515.1 are the same night)
    splitra = np.split(testdf['ra'].values, np.where(np.abs(np.diff(testdf['midPointTai'].values)) >= 0.5)[0] + 1) # split right ascension BY NIGHT
    splitdec = np.split(testdf['decl'].values, np.where(np.abs(np.diff(testdf['midPointTai'].values)) >= 0.5)[0] + 1) # split declination BY NIGHT
    ras = [list(x) for x in splitra if len(x) > 1] # turn above into lists for manipulation purposes
    decs = [list(x) for x in splitdec if len(x) > 1]
    nights = [list(x) for x in splits if len(x) > 1]
    obspernight = [len(ns) for ns in nights]



    # loop over each night and test for tracklets
    for n, night in enumerate(nights):
        if obspernight[n] < 2: # make sure more than 2 observations in a night
            pass
        else:
            for i in np.arange(len(night)):
                for j in np.arange(len(night)):
                    if j <= i: # make sure not comparing same night with itself, or night with night in the past
                        pass
                    else:
                        diff = night[j] - night[i]
                        if diff >= 0 and diff <= 90/(60*24): # make sure time diff <= 90mins
                            coord1 = SkyCoord([ras[n][i]], [decs[n][i]], unit=u.deg) 
                            coord2 = SkyCoord([ras[n][j]], [decs[n][j]], unit=u.deg)
                            sep = coord1.separation(coord2).arcsecond
                            if sep >= 0.5: # make sure spatial separation >= 0.5"
                                # append tracklet to list of confirmed found tracklets
                                tracklet = dict(
                                    no=no,
                                    ssObjectId=ssObjects[a],
                                    ra1=ras[n][i],
                                    dec1=decs[n][i],
                                    ra2=ras[n][j],
                                    dec2=decs[n][j],
                                    mjd1=night[i],
                                    mjd2=night[j]
                                )
                                tracklets.append(tracklet)
                                no += 1

# create dataframe of found tracklets 
linkdf = pd.DataFrame(tracklets)

# same data preparing step as above
nameLen = linkdf['ssObjectId'].str.len().max()
links = linkdf.to_records(
    index=False,
    column_dtypes=dict(no='f8', ssObjectId=f'a{nameLen}', ra1='f8', dec1='f8', ra2='f8', dec2='f8', mjd1='f8', mjd2='f8')
)

# split record array by object
i = np.argsort(links['ssObjectId'], kind='stable')
ssObjects, idx = np.unique(links['ssObjectId'][i], return_index=True)
objSplits = np.split(i, idx[1:])

noLinkages = []
links = []

# START TRACKLET LINKING LOOP
# manually check which tracklet pairs are within 15 days of each other
for a in range(len(ssObjects)):
    testdf = linkdf.loc[objSplits[a],:]
    testdf = testdf.sort_values(by=['mjd1'])
    # display(HTML(testdf.to_html()))

    # loop over each tracklet now
    for n, i in enumerate(np.arange(len(testdf))):
        for m, j in enumerate(np.arange(len(testdf))):
            if j <= i: # ensure we're not linking self with self or self with temporally past tracklets
                pass
            else:
                diff = testdf.iloc[j,:]['mjd2'] - testdf.iloc[i,:]['mjd1'] # calculate time difference between ith and jth tracklet
                if diff > 15: # if the difference is above 15 days, stop finding time differences and use next mjd
                    idxs = np.arange(n, m) # create array of indexes of all mjds from ith trackle to jth tracklet
                    if idxs.size != 0:
                        floors = [np.floor(x) for x in testdf.iloc[idxs,:]['mjd1']] # <-- makes sure that the tracklets are defined on seperate nights to avoid linking 3 tracklets on e.g. nights 1, 5.2, and 5.5
                        if len(np.unique(floors)) >= 3: # if there are tracklets on 3 unique nights, append them to a list of confirmed linked tracklets
                            links.append(dict(
                                ssObjectId=testdf.iloc[i,:]['ssObjectId'],
                                ra1=testdf.iloc[i,:]['ra1'],
                                dec1=testdf.iloc[i,:]['dec1'],
                                ra2=testdf.iloc[j-1,:]['ra2'],
                                dec2=testdf.iloc[j-1,:]['dec2'],
                                mjd1=testdf.iloc[i,:]['mjd1'],
                                mjd2=testdf.iloc[j-1,:]['mjd2']
                            ))
                    break
    

# create dataframe of confirmed linked tracklets
confirmeddf = pd.DataFrame(links)

# same as above two times
nameLen = confirmeddf['ssObjectId'].str.len().max()
links = confirmeddf.to_records(
    index=False,
    column_dtypes=dict(no='f8', ssObjectId=f'a{nameLen}', ra1='f8', dec1='f8', ra2='f8', dec2='f8', mjd1='f8', mjd2='f8')
)

# split record array by object
i = np.argsort(links['ssObjectId'], kind='stable')
ssObjects, idx = np.unique(links['ssObjectId'][i], return_index=True)
objSplits = np.split(i, idx[1:])

noLinksPerObj = []

# final loop that filters out tracklets such that if there are more than one tracklet on
# a given MJD, assume that there is just one single tracklet for that night
for a in range(len(ssObjects)):
    testdf = confirmeddf.loc[objSplits[a],:]
    testdf = testdf.sort_values(by=['mjd1'])
    display(HTML(testdf.to_html()))
    noLinksPerObj.append(len(np.unique(np.floor(testdf['mjd1'].values)))) # this is the step that does it, floor the mjd date and then find the length of unique mjds 


In [ ]:
# display each object and the number of brute force linked tracklets it has
bruteDF = pd.DataFrame({'objs':ssObjects, 'links':noLinksPerObj})
bruteDF

In [ ]:
# predicted number of links from miniDifi output (4th column of each row of obj = linkObservations(...) 
# from PPLinkingFilter)
minidifiLinks = pd.read_csv('/Users/josephmurtagh/Documents/PhD/minidifi/sorcha_minidifi/noLinks.csv', header=None)
minidifiObjs = pd.read_csv('/Users/josephmurtagh/Documents/PhD/minidifi/sorcha_minidifi/objs.csv')
minidifiDF = pd.DataFrame({'objs':minidifiObjs['0'], 'links':minidifiLinks[0]})
minidifiDF